### review

DataFrame에 대해 function을 적용하고 싶다면 apply()를 활용. 이 메서드는 첫 인자로 함수를 필수 값으로 받는다. 두번째 인자로 axis 사용 가능. 0이 디폴트  
axis 0 or index인 경우 각 column에 대해 함수 적용  
axis 1 or columns인 경우 각 row에 대해 함수 적용

DataFrame.aplly(func, axis=0, raw=False, result_type=None, args=(), **kwargs)

In [12]:
import pandas as pd
import numpy as np
# titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")

In [13]:
import seaborn as sns
titanic = sns.load_dataset("titanic")

Numpy의 np.sqrt 사용하여 적용 되. 각 요소마다 적용되는 함수(universal function)로 이 경우에는 np.sqrt(df)와 동일한 경과 가져옴

In [14]:
df = pd.DataFrame([[4, 9]] * 3, columns=["A", "B"])
df

,A,B
0,4,9
1,4,9
2,4,9


In [15]:
df.apply(np.sqrt)

,A,B
0,2.0,3.0
1,2.0,3.0
2,2.0,3.0


In [16]:
np.sqrt(df)

,A,B
0,2.0,3.0
1,2.0,3.0
2,2.0,3.0


차원 축소 함수(reducing function)인 sum 사용. axis의 값에따라 축소되는 방향이 달라져 서로 결과가 다름

In [17]:
df.apply(np.sum, axis =0) # column별 집계

A    12
B    27
dtype: int64

In [18]:
df.apply(np.sum, axis =1) # row별 집계

0    13
1    13
2    13
dtype: int64

함수의 return이 column마다 리스트를 반환하면 DataFrame의 결과를 얻을수있다. 함수의 return이 row마다 리스트를 반환하면 각 row마다 리스트를 하나의 값으로 취급하는 series타입의 결과가 나옴

In [19]:
df.apply(lambda x: [1, 2], axis=0)

,A,B
0,1,1
1,2,2


In [20]:
df.apply(lambda x: [1, 2], axis=1)

0    [1, 2]
1    [1, 2]
2    [1, 2]
dtype: object

In [21]:
df.apply(lambda x: [1, 2], axis=1, result_type="expand")

,0,1
0,1,2
1,1,2
2,1,2


In [22]:
df.apply(lambda x: pd.Series([1, 2], index=["foo", "bar"])

SyntaxError: unexpected EOF while parsing (1021486507.py, line 1)

result_type = "broadcast"를 인수로 전달하면 동일한 shape의 결과를 보당합니다. 함수로부터 반환되는게 리스트인지 스칼라인지에 상관없이 axis 방향으로 브로드캐스트합니다. 결과의 column label은 본래의 column label을 유지합니다.

In [23]:
df.apply(lambda x:[1, 2], axis=1, result_type="broadcast")

,A,B
0,1,2
1,1,2
2,1,2


result_type="broadcast"를 인수로 전달할 떄 함수로부터 return되는 값이 기존 shape으로 브로드캐스틓ㄹ 수 없는 shape이라면 valueError 발생

In [24]:
df.apply(lambda x:[1, 2, 3], axis=1, result_type="broadcast")

ValueError: cannot broadcast result

In [25]:
df3 = pd.DataFrame({
    "A": [1, 3, 4, 3, 4],
    "B": [2, 3, 1, 2, 3],
    "C": [1, 5, 2, 4, 4]
})
df3

,A,B,C
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


apply() 메서드. 예를 들어 cloumn마다의 최대값과 최소값의 차이를 구하고 싶으면 다음과 같은 lamda함수 넣으면 된다.

In [26]:
df3.apply(lambda x: x.max() - x.min()) # default: axis=0

A    3
B    2
C    4
dtype: int64

In [27]:
df3.apply(lambda x: x.max() - x.min(), axis=1)

0    1
1    2
2    3
3    2
4    1
dtype: int64

In [28]:
df3.apply(pd.value_counts) # 개수가 소수점으로 나오는 이유는 NaN이 float형이라서

,A,B,C
1,1.0,1.0,1.0
2,NaN,2.0,1.0
3,2.0,2.0,NaN
4,2.0,NaN,2.0
5,NaN,NaN,1.0


타이타닉호의 승객 중 나이 20살을 기준으로 성인과 미성년자를 구별하는 label cloumn

In [29]:
# titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")

In [30]:
titanic.count()

survived       891
pclass         891
sex            891
age            714
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
dtype: int64

In [31]:
titanic["adult/child"] = titanic.apply(lambda r: "adult" if r.age >= 20 else "child", axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,adult
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,adult
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,adult


연습문제: 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 column인 category1 열을 만들어보세요.
1) 20살이 넘으면 성별을 그대로 사용
2) 20살 미만이면 성별에 관계없이 child

In [32]:
titanic["category1"] = titanic.apply(lambda r: "child" if r.age<20 else r.sex, axis=1)
titanic.tail(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child,category1
881,0,3,male,33.0,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True,adult,male
882,0,3,female,22.0,0,0,10.5167,S,Third,woman,False,NaN,Southampton,no,True,adult,female
883,0,2,male,28.0,0,0,10.5000,S,Second,man,True,NaN,Southampton,no,True,adult,male
884,0,3,male,25.0,0,0,7.0500,S,Third,man,True,NaN,Southampton,no,True,adult,male
885,0,3,female,39.0,0,5,29.1250,Q,Third,woman,False,NaN,Queenstown,no,False,adult,female
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,adult,male
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,child
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child,female
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,adult,male
890,0,3,male,32.0,0,0,7.7500,Q,Third,man,True,NaN,Queenstown,no,True,adult,male


In [33]:
titanic["category1_1"] = titanic.apply(lambda x: x["sex"] if x["age"] >= 20 else "child", axis=1)

In [34]:
titanic.tail(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child,category1,category1_1
881,0,3,male,33.0,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True,adult,male,male
882,0,3,female,22.0,0,0,10.5167,S,Third,woman,False,NaN,Southampton,no,True,adult,female,female
883,0,2,male,28.0,0,0,10.5000,S,Second,man,True,NaN,Southampton,no,True,adult,male,male
884,0,3,male,25.0,0,0,7.0500,S,Third,man,True,NaN,Southampton,no,True,adult,male,male
885,0,3,female,39.0,0,5,29.1250,Q,Third,woman,False,NaN,Queenstown,no,False,adult,female,female
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,adult,male,male
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,child,child
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child,female,child
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,adult,male,male
890,0,3,male,32.0,0,0,7.7500,Q,Third,man,True,NaN,Queenstown,no,True,adult,male,male


fillna() 메서드를 사용하여 NaN 값을 원하는 값으로 바꿀 수 있따. 첫 인자로 NaN을 변경하고자 하는 값을 전달하면 된다.
DataFrame.fillna(value=None, *, method=None, axis=None, inplace=False, limit=None, downcast=None)

In [35]:
df = pd.DataFrame([[np.nan ,2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, np.nan],
                   [np.nan, 3, np.nan, 4]],
                  columns = list("ABCD"))
df

,A,B,C,D
0,NaN,2.0,NaN,0.0
1,3.0,4.0,NaN,1.0
2,NaN,NaN,NaN,NaN
3,NaN,3.0,NaN,4.0


In [36]:
df.fillna(0)

,A,B,C,D
0,0.0,2.0,0.0,0.0
1,3.0,4.0,0.0,1.0
2,0.0,0.0,0.0,0.0
3,0.0,3.0,0.0,4.0


fillna() 메서드 value 값으로 column label을 key로 갖는 딕셔너리를 전달할 수 있다. 그러면 column마다 NaN을 대치하는 값을 각각 다르게 지정할 수 있다.

In [37]:
values = {"A": 0, "B":1, "C":2, "D":3}
df.fillna(value=values)

,A,B,C,D
0,0.0,2.0,2.0,0.0
1,3.0,4.0,2.0,1.0
2,0.0,1.0,2.0,3.0
3,0.0,3.0,2.0,4.0


fillna() 메서드에 limit 키워드 인자에 숫자를 전달하여 그 숫자만큼 column마다 변경 횟수를 제한할 수 있따.

In [38]:
values = {"A": 0, "B":1, "C":2, "D":3}
df.fillna(value=values, limit=1)

,A,B,C,D
0,0.0,2.0,2.0,0.0
1,3.0,4.0,NaN,1.0
2,NaN,1.0,NaN,3.0
3,NaN,3.0,NaN,4.0


fillna() 메서드에 DataFrame을 value로 전달해서 NaN값을 대체할 수 있다. 다만 column label과 row index가 일치하지 않으면 적용되지 않는다.

In [39]:
df2 = pd.DataFrame(np.zeros((3, 4)), columns=list("ABCE"))
df.fillna(df2)

,A,B,C,D
0,0.0,2.0,0.0,0.0
1,3.0,4.0,0.0,1.0
2,0.0,0.0,0.0,NaN
3,NaN,3.0,NaN,4.0


연습문제: 타이타닉호의 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이값이 되도록 titanic DataFrame을 고쳐라

isna() 메서드 써서 NaN 값인 것 찾을 수 있다.

In [40]:
titanic.count()

survived       891
pclass         891
sex            891
age            714
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
adult/child    891
category1      891
category1_1    891
dtype: int64

In [41]:
index_age_is_nan = titanic[titanic["age"].isna()].index
print(index_age_is_nan)

Int64Index([  5,  17,  19,  26,  28,  29,  31,  32,  36,  42,
            ...
            832, 837, 839, 846, 849, 859, 863, 868, 878, 888],
           dtype='int64', length=177)


In [42]:
titanic.fillna({'age': round(titanic['age'].mean(), 1)}, inplace=True)

In [43]:
titanic.iloc[index_age_is_nan]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child,category1,category1_1
5,0,3,male,29.7,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True,child,male,child
17,1,2,male,29.7,0,0,13.0000,S,Second,man,True,NaN,Southampton,yes,True,child,male,child
19,1,3,female,29.7,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True,child,female,child
26,0,3,male,29.7,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True,child,male,child
28,1,3,female,29.7,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True,child,female,child
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0,3,male,29.7,0,0,7.2292,C,Third,man,True,NaN,Cherbourg,no,True,child,male,child
863,0,3,female,29.7,8,2,69.5500,S,Third,woman,False,NaN,Southampton,no,False,child,female,child
868,0,3,male,29.7,0,0,9.5000,S,Third,man,True,NaN,Southampton,no,True,child,male,child
878,0,3,male,29.7,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True,child,male,child


In [44]:
values = titanic["age"].mean()
print(values)
titanic["age"].fillna(values)
titanic

29.69929292929302


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child,category1,category1_1
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,adult,male,male
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,adult,female,female
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,adult,female,female
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,adult,female,female
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,adult,male,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,adult,male,male
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,child,child
888,0,3,female,29.7,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child,female,child
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,adult,male,male


In [45]:
titanic.count()

survived       891
pclass         891
sex            891
age            891
sibsp          891
parch          891
fare           891
embarked       889
class          891
who            891
adult_male     891
deck           203
embark_town    889
alive          891
alone          891
adult/child    891
category1      891
category1_1    891
dtype: int64

astype() 메서드로 column의 자료형을 바꾸는것도 가능

In [46]:
d = {'col1':[1, 2], "col2":[3, 4]}
df = pd.DataFrame(data=d)
df.dtypes

col1    int64
col2    int64
dtype: object

In [47]:
df

,col1,col2
0,1,3
1,2,4


In [48]:
df.astype('int32').dtypes

col1    int32
col2    int32
dtype: object

In [49]:
# df.astype({'col1':[1, 2], 'col2':[3, 4]})

In [50]:
df.astype({'col1':'int32'}).dtypes
# {column label:dtype}

col1    int32
col2    int64
dtype: object

연습문제: 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 column인 category2 column을 만들어보세요. category2 카테고리는 다음과 같이 정의됩니다.

1) 성별을 나타내는 문자열 male또는 female로 시작
2) 성별을 나타내는 문자열 뒤에 나이를 나타내는 문자열이 옵니다.
3) 예를 들어 27살 남성은 male27.0 값이 됩니다.

In [51]:
titanic['category2'] = titanic.sex + titanic.age.astype(str)
titanic[['age', 'category2']]

,age,category2
0,22.0,male22.0
1,38.0,female38.0
2,26.0,female26.0
3,35.0,female35.0
4,35.0,male35.0
...,...,...
886,27.0,male27.0
887,19.0,female19.0
888,29.7,female29.7
889,26.0,male26.0


In [52]:
titanic['category2'] = titanic["sex"] + titanic['age'].astype("str")
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child,category1,category1_1,category2
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,adult,male,male,male22.0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,adult,female,female,female38.0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,adult,female,female,female26.0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,adult,female,female,female35.0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,adult,male,male,male35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,adult,male,male,male27.0
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,child,child,female19.0
888,0,3,female,29.7,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child,female,child,female29.7
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,adult,male,male,male26.0


dataframe 실수 값을 카테고리 값으로 변환
실수 값을 크기 기준으로 하여 카테고리 값으로 변환하고 싶을 때는 다음과 같은 명령을 사용.

- cut: 실수 값의 경계선을 지정하는 경우
    - x = 1차원 형태의 배열 형태가 온다.
    - bins = int, 스칼라를 요소로 갖는 시퀀스가 옵니다.
- qcut: 개수가 똑같은 구간으로 나누는 경우(분위수)
    - x = 1d ndarray 혹은 Series
    - q = int 혹은 분위수를 나타내는 1.이하의 실수를 요소로 갖는 list (ex) [0, .25, .5, .75, 1.])

cut 명령을 사용하면 실수값을 다음처럼 카테고리 값으로 바꿀 수 있다. bins인수는 카테고리를 나누는 기준값이 된다. 영역을 넘는 값은 NaN으로 처리

In [53]:
ages = [0, 2, 10, 21, 23, 37, 31, 61, 20, 41, 32, 101]

In [54]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "장년", "중년", "노년"]
cats = pd.cut(ages, bins, labels=labels)
cats

[NaN, '미성년자', '미성년자', '청년', '청년', ..., '중년', '미성년자', '장년', '장년', NaN]
Length: 12
Categories (5, object): ['미성년자' < '청년' < '장년' < '중년' < '노년']

cut() 명령이 반환하는 값은 Categorical 클래스 객체. 이 객체는 속성으로 label 문자열을, codes 속성으로 정수로 인코딩한 카테고리 값을 가짐

In [55]:
type(cats)

pandas.core.arrays.categorical.Categorical

In [56]:
cats.categories

Index(['미성년자', '청년', '장년', '중년', '노년'], dtype='object')

In [57]:
cats.codes

array([-1,  0,  0,  1,  1,  2,  2,  3,  0,  2,  2, -1], dtype=int8)

In [58]:
df4 = pd.DataFrame(ages, columns=["ages"])
df4["age_cat"] = pd.cut(df4.ages, bins, labels = labels)
df4

,ages,age_cat
0,0,NaN
1,2,미성년자
2,10,미성년자
3,21,청년
4,23,청년
5,37,장년
6,31,장년
7,61,중년
8,20,미성년자
9,41,장년


In [59]:
df4.dtypes

ages          int64
age_cat    category
dtype: object

In [60]:
df4["age_cat"].astype(str) + df4['ages'].astype(str)

0       nan0
1      미성년자2
2     미성년자10
3       청년21
4       청년23
5       장년37
6       장년31
7       중년61
8     미성년자20
9       장년41
10      장년32
11    nan101
dtype: object

qcut() 명령은 구간 경계선을 지정하지 않고 분위수와 같이 데이터 개수가 같도록 구간을 나눕니다. 예를 들어 다음 코드는 1000개의 데이터를 4개의 구간으로 나누는데 각 구간은 250개씩의 데이터를 가집니다.

In [61]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4, labels=["Q1", "Q2", "Q3", "Q4"])
cats

['Q4', 'Q2', 'Q2', 'Q2', 'Q4', ..., 'Q2', 'Q2', 'Q3', 'Q2', 'Q1']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [62]:
pd.value_counts(cats)

Q1    250
Q2    250
Q3    250
Q4    250
dtype: int64

연습문제: 타이타닉호 승객을 '미성년자', '청년', '장년', '중년', '노년' 나이 그룹으로 나눕니다. 그리고 각 나이 그룹의 승객 비율을 구합니다. 비율의 전체 합은 1이 되어야 합니다.

In [63]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "장년", "중년", "노년"]
titanic['age_group'] = pd.cut(titanic['age'], bins, labels = labels)
(titanic['age_group'].value_counts())/(titanic['age_group'].value_counts().sum())

청년      0.464082
장년      0.274800
미성년자    0.188141
중년      0.067275
노년      0.005701
Name: age_group, dtype: float64

In [64]:
titanic['age_group'].value_counts().sum()

877

In [65]:
titanic['age_group'].value_counts(normalize=True)

청년      0.464082
장년      0.274800
미성년자    0.188141
중년      0.067275
노년      0.005701
Name: age_group, dtype: float64

연습문제: 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 column인 category3을 만들어라. category3 카테고리는 다음과 같이 정의됩니다.
1) 20살 미만이면 성별에 관계 없이 "미성년자"라고 합니다.
2) 20살 이상이면 나이에 따라 "청년", "장년", "중년", "노년"을 구분하고 그 뒤에 성별을 나타내는 "남성", "여성"을 붙인다.

In [66]:
titanic['category3'] = titanic.apply(lambda i: "미성년자" if i["age"] <20 else i["age_group"]
                                     + ("남성" if i["sex"] == "male" else "여성"), axis=1)
titanic[["age_group", "age", "sex", "category3"]]

,age_group,age,sex,category3
0,청년,22.0,male,청년남성
1,장년,38.0,female,장년여성
2,청년,26.0,female,청년여성
3,장년,35.0,female,장년여성
4,장년,35.0,male,장년남성
...,...,...,...,...
886,청년,27.0,male,청년남성
887,미성년자,19.0,female,미성년자
888,청년,29.7,female,청년여성
889,청년,26.0,male,청년남성


In [67]:
titanic[(titanic['age'] <1) | (titanic['age'] > 100)][["age_group", "age", "sex", "category3"]]

,age_group,age,sex,category3
78,NaN,0.83,male,미성년자
305,NaN,0.92,male,미성년자
469,NaN,0.75,female,미성년자
644,NaN,0.75,female,미성년자
755,NaN,0.67,male,미성년자
803,NaN,0.42,male,미성년자
831,NaN,0.83,male,미성년자


In [68]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,...,deck,embark_town,alive,alone,adult/child,category1,category1_1,category2,age_group,category3
0,0,3,male,22.0,1,0,7.2500,S,Third,man,...,NaN,Southampton,no,False,adult,male,male,male22.0,청년,청년남성
1,1,1,female,38.0,1,0,71.2833,C,First,woman,...,C,Cherbourg,yes,False,adult,female,female,female38.0,장년,장년여성
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,...,NaN,Southampton,yes,True,adult,female,female,female26.0,청년,청년여성
3,1,1,female,35.0,1,0,53.1000,S,First,woman,...,C,Southampton,yes,False,adult,female,female,female35.0,장년,장년여성
4,0,3,male,35.0,0,0,8.0500,S,Third,man,...,NaN,Southampton,no,True,adult,male,male,male35.0,장년,장년남성
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,...,NaN,Southampton,no,True,adult,male,male,male27.0,청년,청년남성
887,1,1,female,19.0,0,0,30.0000,S,First,woman,...,B,Southampton,yes,True,child,child,child,female19.0,미성년자,미성년자
888,0,3,female,29.7,1,2,23.4500,S,Third,woman,...,NaN,Southampton,no,False,child,female,child,female29.7,청년,청년여성
889,1,1,male,26.0,0,0,30.0000,C,First,man,...,C,Cherbourg,yes,True,adult,male,male,male26.0,청년,청년남성


In [69]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "장년", "중년", "노년"]
titanic_age = pd.cut(titanic['age'], bins, labels = labels)
titanic_age_rate = titanic_age / titanic_age.sum()

TypeError: 'Categorical' with dtype category does not support reduction 'sum'

set_index: 기존의 row 인덱스를 제거하고 데이터 column 중 하나를 인덱스로 설정한다.
reset_index: 기존의 row 인덱스를 제거하고 인덱스를 데이터 열로 추가한다.

In [71]:
np.random.seed(0)
df1 = pd.DataFrame(np.vstack([list('ABCDE'),
                              np.round(np.random.rand(3, 5), 2)]).T,
                   columns=["C1", "C2", "C3", "C4"])
df1

,C1,C2,C3,C4
0,A,0.55,0.65,0.79
1,B,0.72,0.44,0.53
2,C,0.6,0.89,0.57
3,D,0.54,0.96,0.93
4,E,0.42,0.38,0.07


In [72]:
df2 = df1.set_index("C1")
df2

,C2,C3,C4
C1,,,
A,0.55,0.65,0.79
B,0.72,0.44,0.53
C,0.6,0.89,0.57
D,0.54,0.96,0.93
E,0.42,0.38,0.07


In [74]:
df2.set_index("C2")

,C3,C4
C2,,
0.55,0.65,0.79
0.72,0.44,0.53
0.6,0.89,0.57
0.54,0.96,0.93
0.42,0.38,0.07


reset_index를 쓰면

In [75]:
df2.reset_index()

,C1,C2,C3,C4
0,A,0.55,0.65,0.79
1,B,0.72,0.44,0.53
2,C,0.6,0.89,0.57
3,D,0.54,0.96,0.93
4,E,0.42,0.38,0.07


인수 drop=True인수로 전달하면 column을 보통의 자료열로 올리는 것이 아니라 그냥 버리게 됨

In [76]:
df2.reset_index(drop=True)

,C2,C3,C4
0,0.55,0.65,0.79
1,0.72,0.44,0.53
2,0.6,0.89,0.57
3,0.54,0.96,0.93
4,0.42,0.38,0.07


연습문제: 5명의 학생의 국어, 영어, 수학 점수를 나타내는 DataFrame을 다음과 같이 만든다.
1) '이름' column을 인덱스로 만들어보세요
2) 1)에서 인덱스로 만든 '이름'을 다시 column으로 복원하세요

In [77]:
score = {
    "이름":["일식", "이식", "삼식", "사식", "오식"],
    "국어":[60, 70, 90, 80, 100],
    "영어":[70, 86, 82, 88, 100],
    "수학":[65, 82, 85, 90, 100]
}
df = pd.DataFrame(score)
df1 = df.set_index("이름")
df1

,국어,영어,수학
이름,,,
일식,60,70,65
이식,70,86,82
삼식,90,82,85
사식,80,88,90
오식,100,100,100


In [78]:
df1.reset_index()

,이름,국어,영어,수학
0,일식,60,70,65
1,이식,70,86,82
2,삼식,90,82,85
3,사식,80,88,90
4,오식,100,100,100


다중 인덱스(multi index)

In [82]:
np.random.seed(0)
df3 = pd.DataFrame(np.round(np.random.randn(5, 4), 2), 
                   columns = [["A", "A", "B", "B"],
                             ["C1", "C2", "C1", "C2"]])
df3

A           B      
     C1    C2    C1    C2
0  1.76  0.40  0.98  2.24
1  1.87 -0.98  0.95 -0.15
2 -0.10  0.41  0.14  1.45
3  0.76  0.12  0.44  0.33
4  1.49 -0.21  0.31 -0.85

In [83]:
df3.columns.names = ["Cidx1", "Cidx2"]
df3

Cidx1     A           B      
Cidx2    C1    C2    C1    C2
0      1.76  0.40  0.98  2.24
1      1.87 -0.98  0.95 -0.15
2     -0.10  0.41  0.14  1.45
3      0.76  0.12  0.44  0.33
4      1.49 -0.21  0.31 -0.85

https://dataengineering20.slack.com/files/U04GR7K8C6T/F04L5JZK0GH/image.pnghttps://dataengineering20.slack.com/files/U04GR7K8C6T/F04L5JZK0GH/image.png

stack() 메서드: column 인게스 -> row 

## slack 보고 하기

In [88]:
np.random.seed(0)
df4 = pd.DataFrame(np.round(np.random.randn(6, 4), 2), 
                   columns = [["A", "A", "B", "B"],
                             ["C", "D", "C", "D"]],
                   index = [["M", "M", "M", "F", "F", "F"],
                           ["id_" + str(i+1) for i in range(3)] * 2])
df4.columns.names = ["Cidx1", "Cidx2"]
df4.index.names = ["Ridx1", "Ridx2"]
df4

Cidx1           A           B      
Cidx2           C     D     C     D
Ridx1 Ridx2                        
M     id_1   1.76  0.40  0.98  2.24
      id_2   1.87 -0.98  0.95 -0.15
      id_3  -0.10  0.41  0.14  1.45
F     id_1   0.76  0.12  0.44  0.33
      id_2   1.49 -0.21  0.31 -0.85
      id_3  -2.55  0.65  0.86 -0.74

stack() 메서드나 unstack() 메서드를 쓰면 column 인덱스를 row 인덱스로 바꾸거나 반대로 row 인덱스를 column 인덱스로 바꿀 수 있습니다.

In [89]:
df4.stack("Cidx1")

Cidx2                 C     D
Ridx1 Ridx2 Cidx1            
M     id_1  A      1.76  0.40
            B      0.98  2.24
      id_2  A      1.87 -0.98
            B      0.95 -0.15
      id_3  A     -0.10  0.41
            B      0.14  1.45
F     id_1  A      0.76  0.12
            B      0.44  0.33
      id_2  A      1.49 -0.21
            B      0.31 -0.85
      id_3  A     -2.55  0.65
            B      0.86 -0.74

In [90]:
df4.unstack("Ridx2")

Cidx1     A                                   B                              
Cidx2     C                 D                 C                 D            
Ridx2  id_1  id_2  id_3  id_1  id_2  id_3  id_1  id_2  id_3  id_1  id_2  id_3
Ridx1                                                                        
F      0.76  1.49 -2.55  0.12 -0.21  0.65  0.44  0.31  0.86  0.33 -0.85 -0.74
M      1.76  1.87 -0.10  0.40 -0.98  0.41  0.98  0.95  0.14  2.24 -0.15  1.45

In [91]:
df4.stack("Cidx1")

Cidx2                 C     D
Ridx1 Ridx2 Cidx1            
M     id_1  A      1.76  0.40
            B      0.98  2.24
      id_2  A      1.87 -0.98
            B      0.95 -0.15
      id_3  A     -0.10  0.41
            B      0.14  1.45
F     id_1  A      0.76  0.12
            B      0.44  0.33
      id_2  A      1.49 -0.21
            B      0.31 -0.85
      id_3  A     -2.55  0.65
            B      0.86 -0.74

In [92]:
df4.stack(0)

Cidx2                 C     D
Ridx1 Ridx2 Cidx1            
M     id_1  A      1.76  0.40
            B      0.98  2.24
      id_2  A      1.87 -0.98
            B      0.95 -0.15
      id_3  A     -0.10  0.41
            B      0.14  1.45
F     id_1  A      0.76  0.12
            B      0.44  0.33
      id_2  A      1.49 -0.21
            B      0.31 -0.85
      id_3  A     -2.55  0.65
            B      0.86 -0.74

In [93]:
df3

Cidx1     A           B      
Cidx2    C1    C2    C1    C2
0      1.76  0.40  0.98  2.24
1      1.87 -0.98  0.95 -0.15
2     -0.10  0.41  0.14  1.45
3      0.76  0.12  0.44  0.33
4      1.49 -0.21  0.31 -0.85

In [94]:
df3[("B", "C1")]

0    0.98
1    0.95
2    0.14
3    0.44
4    0.31
Name: (B, C1), dtype: float64

loc 인덱서를 사용하는 경우에도 마찬가지로 튜플을 사용해서 인덱싱해야 한다.

In [95]:
df3

Cidx1     A           B      
Cidx2    C1    C2    C1    C2
0      1.76  0.40  0.98  2.24
1      1.87 -0.98  0.95 -0.15
2     -0.10  0.41  0.14  1.45
3      0.76  0.12  0.44  0.33
4      1.49 -0.21  0.31 -0.85

In [97]:
df3.loc[0, ("B", "C1")]

0.98

In [98]:
df3.loc[0, ("B", "C1")] = 100
df3

Cidx1     A             B      
Cidx2    C1    C2      C1    C2
0      1.76  0.40  100.00  2.24
1      1.87 -0.98    0.95 -0.15
2     -0.10  0.41    0.14  1.45
3      0.76  0.12    0.44  0.33
4      1.49 -0.21    0.31 -0.85

단 iloc 인덱서를 사용하는 경우에는 튜플 형태의 다중 인덱스를 사용할 수 없다. iloc은 가장 안쪽에 접근

In [99]:
df3.iloc[0, 2]

100.0

만약 하나의 레벨 값만 넣으면 다중 인덱스 중에서 가장 상위의 값을 지정한 것으로 반환한다.

In [101]:
df3["A"]

Cidx2,C1,C2
0,1.76,0.40
1,1.87,-0.98
2,-0.10,0.41
3,0.76,0.12
4,1.49,-0.21


DataFrame 다중 인덱스가 있는 경우의 인덱싱

In [103]:
df4

Cidx1           A           B      
Cidx2           C     D     C     D
Ridx1 Ridx2                        
M     id_1   1.76  0.40  0.98  2.24
      id_2   1.87 -0.98  0.95 -0.15
      id_3  -0.10  0.41  0.14  1.45
F     id_1   0.76  0.12  0.44  0.33
      id_2   1.49 -0.21  0.31 -0.85
      id_3  -2.55  0.65  0.86 -0.74

In [105]:
df4.loc[:, ("A", "C")]

Ridx1  Ridx2
M      id_1     1.76
       id_2     1.87
       id_3    -0.10
F      id_1     0.76
       id_2     1.49
       id_3    -2.55
Name: (A, C), dtype: float64

In [104]:
df4.loc[("M", "id_1"), ("A", "C")]

1.76

In [106]:
df4.loc[("M", "id_1"), :]

Cidx1  Cidx2
A      C        1.76
       D        0.40
B      C        0.98
       D        2.24
Name: (M, id_1), dtype: float64

In [107]:
df4.loc[("All", "All"), :] = df4.sum()
df4

Cidx1           A           B      
Cidx2           C     D     C     D
Ridx1 Ridx2                        
M     id_1   1.76  0.40  0.98  2.24
      id_2   1.87 -0.98  0.95 -0.15
      id_3  -0.10  0.41  0.14  1.45
F     id_1   0.76  0.12  0.44  0.33
      id_2   1.49 -0.21  0.31 -0.85
      id_3  -2.55  0.65  0.86 -0.74
All   All    3.23  0.39  3.68  2.28

In [109]:
df4.loc["M"]

Cidx1     A           B      
Cidx2     C     D     C     D
Ridx2                        
id_1   1.76  0.40  0.98  2.24
id_2   1.87 -0.98  0.95 -0.15
id_3  -0.10  0.41  0.14  1.45

### DataFrame 다중 인덱스가 있는 경우의 인덱싱

특정 레벨의 모든 인덱스 값을 인덱싱할 때는 슬라이스 사용. 다만 다중 인덱스의 튜플 내에서는 콜론(:) 즉 슬라이스 기호를 사용할 수 없고 대신 slice(None)값을 사용해야 한다.

In [110]:
df4.loc[("M", slice(None)), :]

Cidx1           A           B      
Cidx2           C     D     C     D
Ridx1 Ridx2                        
M     id_1   1.76  0.40  0.98  2.24
      id_2   1.87 -0.98  0.95 -0.15
      id_3  -0.10  0.41  0.14  1.45

In [112]:
df4.loc[(slice(None), "id_1"), :]

Cidx1           A           B      
Cidx2           C     D     C     D
Ridx1 Ridx2                        
M     id_1   1.76  0.40  0.98  2.24
F     id_1   0.76  0.12  0.44  0.33

다중 인덱스의 인게스 순서를 바꾸고 싶으면 swaplevel 명령을 사용합니다.
swaplevel(i, j, axis)
i와 j는 교환하고자 하는 인덱스  label(혹은 인덱스 번호)이고 axis는 0일 때 row 인덱스, 1일 때 column 인덱스를 뜻합니다. 디폴트는 행 인덱스

In [113]:
df6 = df4.swaplevel("Cidx1", "Cidx2", 1)
df6

Cidx2           C     D     C     D
Cidx1           A     A     B     B
Ridx1 Ridx2                        
M     id_1   1.76  0.40  0.98  2.24
      id_2   1.87 -0.98  0.95 -0.15
      id_3  -0.10  0.41  0.14  1.45
F     id_1   0.76  0.12  0.44  0.33
      id_2   1.49 -0.21  0.31 -0.85
      id_3  -2.55  0.65  0.86 -0.74
All   All    3.23  0.39  3.68  2.28

In [114]:
df5 = df4.swaplevel("Ridx1", "Ridx2")
df5

Cidx1           A           B      
Cidx2           C     D     C     D
Ridx2 Ridx1                        
id_1  M      1.76  0.40  0.98  2.24
id_2  M      1.87 -0.98  0.95 -0.15
id_3  M     -0.10  0.41  0.14  1.45
id_1  F      0.76  0.12  0.44  0.33
id_2  F      1.49 -0.21  0.31 -0.85
id_3  F     -2.55  0.65  0.86 -0.74
All   All    3.23  0.39  3.68  2.28

다중 인덱스가 있는 DataFrame을 sort_index로 정렬할 때는 level 인수를 사용하여 어떤 인덱스를 기준으로 정렬하는지 알려주어야 합니다.

In [115]:
df5.sort_index(level=0)

Cidx1           A           B      
Cidx2           C     D     C     D
Ridx2 Ridx1                        
All   All    3.23  0.39  3.68  2.28
id_1  F      0.76  0.12  0.44  0.33
      M      1.76  0.40  0.98  2.24
id_2  F      1.49 -0.21  0.31 -0.85
      M      1.87 -0.98  0.95 -0.15
id_3  F     -2.55  0.65  0.86 -0.74
      M     -0.10  0.41  0.14  1.45

In [117]:
df6.sort_index(axis=1, level=0)

Cidx2           C           D      
Cidx1           A     B     A     B
Ridx1 Ridx2                        
M     id_1   1.76  0.98  0.40  2.24
      id_2   1.87  0.95 -0.98 -0.15
      id_3  -0.10  0.14  0.41  1.45
F     id_1   0.76  0.44  0.12  0.33
      id_2   1.49  0.31 -0.21 -0.85
      id_3  -2.55  0.86  0.65 -0.74
All   All    3.23  3.68  0.39  2.28

### DataFrame 합성
pandas는 두개 이상의 DataFrame을 하나로 합치는 데이터 병합(merge)이나 연결(concatenate)을 지원합니다.

merge() 함수는 두 데이터 프레임의 공통 column 혹은 인덱스를 기준으로 두 개의 테이블을 합친다. 이 때 기준이 되는 column, row의 데이터를 키(key)라고 한다.

In [118]:
df1 = pd.DataFrame({
    '고객번호':[1001, 1002, 1003, 1004, 1005, 1006, 1007],
    '이름': ['둘리', '도우너', '또치', '길동', '희동', '마이콜', '영희']
}, columns = ['고객번호', '이름'])
df1

,고객번호,이름
0,1001,둘리
1,1002,도우너
2,1003,또치
3,1004,길동
4,1005,희동
5,1006,마이콜
6,1007,영희


In [120]:
df2 = pd.DataFrame({
    '고객번호':[1001, 1001, 1005, 1006, 1008, 1001],
    '금액': [10000, 20000, 15000, 5000, 100000, 30000]
}, columns = ['고객번호', '금액'])
df2

,고객번호,금액
0,1001,10000
1,1001,20000
2,1005,15000
3,1006,5000
4,1008,100000
5,1001,30000


merge 함수로 위의 두 DataFrame df1, df2를 합치면 공통 column인 고객번호 column을 기준으로 데이터를 찾아서 합친다. 이 떄 기본적으로는 양쪽 DataFrame에 모두 키가 존재하는 데이터만 보여주는 inner join방식을 사용합니다.

In [121]:
pd.merge(df1, df2)

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1001,둘리,30000
3,1005,희동,15000
4,1006,마이콜,5000


outer join 방식은 키 값이 한쪽에만 있어도 데이터 보여줍니다.

In [122]:
pd.merge(df1, df2, how='outer')

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN
9,1008,NaN,100000.0


left는 첫 번째 인수를 기준으로, right는 2번째 인수 기준으로 DataFrame의 키 값을 모두 보여줍니다.

In [123]:
pd.merge(df1, df2, how='left')

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN


In [124]:
pd.merge(df1, df2, how='right')

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1005,희동,15000
3,1006,마이콜,5000
4,1008,NaN,100000
5,1001,둘리,30000
